In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models.yolov4 import CSPDarknet53_SPP_PAN
from models.common import *
from utility.loss_function import *
from utility.utilities import *
from utility.boxes import *
from utility.dataset import *
from utility.dataset_api import *
from utility.display import *

In [ ]:
EPOCHS = 1

In [3]:
device = torch.device('cpu')

ds = CocoDatasetAPITrain()
dl = load_dataloader(ds, batch_size=4)

modelTest = CSPDarknet53_SPP_PAN()
modelTest = modelTest.to(device=device)

optimiser = optim.Adam(modelTest.parameters())

loading annotations into memory...
Done (t=0.53s)
creating index...
index created!


In [6]:
for epoch in EPOCHS:
    for i, batch in enumerate(dl):
        images, targets = batch
        images = images.to(device=device)
        targets = targets.to(device=device)
        
        preds = modelTest(images)[-1]
        
        preds = split_output(preds, device)
        
        preds[0][:,:,:,:,0:2] = ACTIVATIONS['sigmoid'](preds[0][:,:,:,:,0:2])
        preds[1] = ACTIVATIONS['sigmoid'](preds[1])
        preds[2] = ACTIVATIONS['sigmoid'](preds[2])
        
        pred_boxes = prediction_to_boxes(preds[0], 's_scale', device)
        
        masks_and_target = build_target(pred_boxes, preds[2], targets, 's_scale', device)
        
        loss = loss_function(preds, masks_and_target, device)
        
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        print('Iteration {} done, loss: {}'.format(i, loss.item()))

        break

Iteration 0 done, loss: 3961.791748046875


In [ ]:
torch.save(model.state_dict(), 'model.pt')